# IMPORT

In [2]:
from __future__ import annotations
from qiskit import QuantumCircuit
from transpiler.qasm2_reader import load_qasm, from_qasm2_to_braket
from bqskit.ext import bqskit_to_qiskit,qiskit_to_bqskit
from qiskit_aer import AerSimulator
from qiskit import transpile
import pennylane as qml
import numpy as np
from braket.devices import LocalSimulator
from braket.circuits import Circuit as braket_Circuit
from transpiler.transpile import transpile_circuit as bqskit_transpiler
from transpiler.transpile import get_noisy_counts
from qiskit.circuit.random import random_circuit

In [5]:
# Create a 4-qubit quantum circuit
# Generate a random 4-qubit circuit with a depth of 5
qc = random_circuit(4,max_operands=2, depth=4, measure=False)

    
qc_qiskit = qc
qc_bqskit=qiskit_to_bqskit(qc)
print(qc)

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

                                  ┌────────────┐┌──────────────┐»
q_0: ─────────────────────────────┤ U1(5.4393) ├┤ Ry(0.027753) ├»
                                  └────────────┘└──────┬───────┘»
q_1: ───────────────────────────────────■──────────────■────────»
     ┌───────────────────────────┐      │                       »
q_2: ┤ U(0.95405,0.58656,4.4116) ├──────┼─────────■─────────────»
     └───────────────────────────┘      │         │U1(6.1464)   »
q_3: ───────────────────────────────────■─────────■─────────────»
                                                                »
«                ┌───┐                       ┌───┐            
«q_0: ───────────┤ Z ├───────────────────────┤ S ├────────────
«     ┌──────────┴───┴───────────┐┌──────────┴───┴───────────┐
«q_1: ┤ U3(5.7409,5.8332,5.2648) ├┤ U3(2.9667,4.3065,4.8197) ├
«     └──────────────────────────┘└────────────┬─────────────┘
«q_2: ─────────────────────────────────────────■──────────────
«                ┌───┐      

In [6]:
# Step 3: Transpile the circuit to MS gates
qc_qiskit_transpiled = bqskit_transpiler(qc_qiskit,device="Aria",optimization_level=2,synthesis_epsilon=1e-4)
print(qc_qiskit_transpiled)

     ┌───────────────┐┌───────────────┐          ┌─────────────┐           »
q_0: ┤ Gpi2(0.84355) ├┤ Gpi2(0.82967) ├──────────┤ Gpi(5.9124) ├───────────»
     └┬──────────────┤└─┬────────────┬┘┌─────────┴─────────────┴──────────┐»
q_1: ─┤ Gpi2(2.7107) ├──┤ Gpi(3.182) ├─┤0                                 ├»
      ├──────────────┤  └────────────┘ │                                  │»
q_2: ─┤ Gpi2(5.0132) ├─────────────────┤  Partialms(5.2241,1.8543,1.5708) ├»
      ├──────────────┤ ┌─────────────┐ │                                  │»
q_3: ─┤ Gpi2(3.9535) ├─┤ Gpi(3.6893) ├─┤1                                 ├»
      └──────────────┘ └─────────────┘ └──────────────────────────────────┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐           ┌─────────────┐                            »
«q_1: ┤ Gpi2(0.8385) ├───────────┤ Gpi(5.1087) ├─────────────────────────

# Convert Qiskit circuit to PennyLane


In [38]:
qml_circuit = qml.from_qiskit(qc_qiskit_transpiled)
print("Transpiled result",np.abs(qml.matrix(qml_circuit, wire_order=list(range(qc_qiskit.num_qubits)))().T[0])**2)#check
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

Transpiled result [4.52992407e-01 1.18251950e-13 1.54418250e-02 6.26847159e-13
 3.08972403e-13 4.19779491e-14 3.15656481e-02 1.03754706e-14
 1.48807062e-01 1.04546264e-13 4.70074402e-02 2.11727637e-13
 3.04185617e-01 6.42433267e-14 1.68391942e-12 4.34054270e-13]
Not transpiled result: [0.45299253 0.14880689 0.         0.30418563 0.01544183 0.04700747
 0.03156564 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


# NOISE

In [35]:
print("Noise 0     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.00,0.000,0.00))
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=list(range(qc_qiskit.num_qubits)))().T[0])**2)
print("Noisy     simulation result:", get_noisy_counts(qc_qiskit_transpiled,1000,0.01,0.001,0.01))#check

Noise 0     simulation result: [0.43847656 0.         0.01171875 0.         0.         0.
 0.03320312 0.         0.15527344 0.         0.0546875  0.
 0.30664062 0.         0.         0.        ]
Noiseless simulation result: [4.52992407e-01 1.18251950e-13 1.54418250e-02 6.26847159e-13
 3.08972403e-13 4.19779491e-14 3.15656481e-02 1.03754706e-14
 1.48807062e-01 1.04546264e-13 4.70074402e-02 2.11727637e-13
 3.04185617e-01 6.42433267e-14 1.68391942e-12 4.34054270e-13]
Noisy     simulation result: {'0000': 382, '0001': 13, '0010': 29, '0011': 5, '0100': 27, '0101': 1, '0110': 24, '1000': 167, '1001': 3, '1010': 50, '1011': 1, '1100': 275, '1101': 5, '1110': 17, '1111': 1}


# Vibratium

In [36]:
bracket_qc=from_qasm2_to_braket("circuit.qasm")


In [37]:
print(bracket_qc)

T  : │        0        │      1       │      2       │      3       │      4       │           5            │      6       │      7       │                        8                        │      9       │      10      │      11      │           12           │      13      │      14      │           15           │      16      │      17      │      18      │           19           │      20      │      21      │     22      │           23           │      24      │      25      │      26      │      27       │ 28  │
                        ┌───────────┐                                                                                                      ┌──────────────────────┐                          ┌───────────┐  ┌────────────┐                ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐ ┌──────────────────────┐ ┌────────────┐ ┌────────────┐ ┌───────────┐ ┌──────────────────────┐ ┌────────────┐ ┌──────────

In [24]:
device = LocalSimulator() # Choose the local simulator
shots = 100000  # Number of times to execute the circuit

result = device.run(bracket_qc, shots=shots).result()
counts = result.measurement_probabilities  # Get the shot results

print(counts)  # Dictionary {bitstring: counts}
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

{'11': 0.35773, '01': 0.123, '10': 0.41512, '00': 0.10415}
Noiseless simulation result: [0.10533638 0.12272559 0.41539821 0.35653982]


# GATE braket TEST

In [17]:
print(braket_Circuit().gpi(0,1))

T  : │    0     │
      ┌────────┐ 
q0 : ─┤ GPi(1) ├─
      └────────┘ 
T  : │    0     │


In [18]:
braket_Circuit().gpi(0,0.445).to_unitary()

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [19]:
phi=0.445
np.array([
            [0, np.exp(-1j * phi)],
            [np.exp(1j * phi), 0]
        ])

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [20]:
braket_Circuit().gpi2(0,0.445).to_unitary()

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [13]:
 # Extract the phase parameter
factor = 1 / np.sqrt(2)  # Normalization factor
exp_pos = -1j * np.exp(1j * phi)
exp_neg = -1j * np.exp(-1j * phi)

factor * np.array([
            [1, exp_neg],
            [exp_pos, 1]
        ])

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [14]:
phi1=0.2455
phi2=0.4157
theta=0.11545
q1=0
q2=1
braket_Circuit().ms(q1, q2, phi1, phi2,theta).to_unitary()

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [15]:
phi0=phi1
phi1=phi2

cos_theta = np.cos(theta/2)
sin_theta = np.sin(theta/2)

e_pos = -1j * np.exp(1j * (phi0 + phi1))
e_neg = -1j * np.exp(-1j * (phi0 + phi1))
e_diff_pos = -1j * np.exp(1j * (phi0 - phi1))
e_diff_neg = -1j * np.exp(-1j * (phi0 - phi1))

np.array([
    [cos_theta, 0, 0, e_neg * sin_theta],
    [0, cos_theta, e_diff_neg * sin_theta, 0],
    [0, e_diff_pos * sin_theta, cos_theta, 0],
    [e_pos * sin_theta, 0, 0, cos_theta]
])

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [16]:
braket_Circuit().zz(0,1,np.pi).to_unitary()

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17-1.j]])

In [13]:
# Load a random quantum circuit (for testing)
qc = random_circuit(num_qubits=4, depth=10, measure=False)

# Choose a backend
backend = AerSimulator()

# Transpile for optimization
transpiled_qc = transpile(qc, backend, optimization_level=3)

In [15]:
print(qc)
print(transpiled_qc)

     ┌────────┐┌────────┐┌────────┐                 ┌────────┐┌────────┐»
q_0: ┤2       ├┤0       ├┤1       ├─■───────────────┤2       ├┤2       ├»
     │        ││        ││        │ │               │        ││        │»
q_1: ┤0       ├┤2       ├┤3       ├─■───────────────┤1       ├┤3       ├»
     │  Rcccx ││  Rcccx ││  Rcccx │ │ ┌────────────┐│  Rcccx ││  Rcccx │»
q_2: ┤3       ├┤1       ├┤0       ├─┼─┤ Ry(3.6933) ├┤0       ├┤1       ├»
     │        ││        ││        │ │ └────────────┘│        ││        │»
q_3: ┤1       ├┤3       ├┤2       ├─■───────────────┤3       ├┤0       ├»
     └────────┘└────────┘└────────┘                 └────────┘└────────┘»
«     ┌───────┐┌────────┐                                    ┌───┐
«q_0: ┤1      ├┤1       ├──────────────────■─────────────────┤ H ├
«     │       ││        │┌───────────────┐ │ ┌──────────────┐└─┬─┘
«q_1: ┤2 Rccx ├┤2       ├┤0              ├─┼─┤1             ├──┼──
«     │       ││  Rcccx ││               │ │ │  Rzx(1.2241) │  │  